In [ ]:
# © 2022. Triad National Security, LLC. All rights reserved.
# This program was produced under U.S. Government contract 89233218CNA000001 for Los Alamos
# National Laboratory (LANL), which is operated by Triad National Security, LLC for the U.S.
# Department of Energy/National Nuclear Security Administration. All rights in the program are
# reserved by Triad National Security, LLC, and the U.S. Department of Energy/National Nuclear
# Security Administration. The Government is granted for itself and others acting on its behalf a
# nonexclusive, paid-up, irrevocable worldwide license in this material to reproduce, prepare
# derivative works, distribute copies to the public, perform publicly and display publicly, and to permit
# others to do so.

In [ ]:
import os
import time
import joblib
import h5py
import random as python_random

import tensorflow as tf
import numpy as np
import scipy as sp

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
print(len(gpus), "Physical GPUs")

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update({'font.size': 16})
mpl.rcParams.update({'font.family': 'serif'})

from utils_data_windows import write_tfrecord_train_valid_data, write_tfrecord_test_data, read_tfrecord_dataset
from utils_data_windows import write_tfrecord_train_valid_data_major_slips, get_mu_slip_loc_data_with_index
from utils_labquake_model import LabquakeModel, LabquakeModel_MS, MuModel
from utils_model_training import train_labquake_model, get_labquake_model
from utils_model_training import test_labquake_model, test_labquake_model_detail

from sklearn.metrics import r2_score

seed_value = 1234
python_random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
verbose = 1
write_tfrecord_data = 0

# Parameters data dict
params_data = {}
params_data['filenames_train']    = ['./Data/p4677_seismic_data_2MPa.hdf5', './Data/p4677_shear_data_2MPa.hdf5']
params_data['train_data_split']   = 0.6
params_data['valid_data_split']   = 0.4

params_data['in_scaler_mode']     = 'standard'
params_data['out_scaler_mode']    = 'minmax'
params_data['sample_freq']        = 1000.0
params_data['unit_time_intvl']    = 0.008
params_data['num_in_time_intvl']  = 32
params_data['num_out_time_intvl'] = 1
params_data['in_time_intvl']      = params_data['unit_time_intvl'] * params_data['num_in_time_intvl']
params_data['out_time_intvl']     = params_data['unit_time_intvl'] * params_data['num_out_time_intvl']
params_data['in_out_time_diff']   = params_data['in_time_intvl']
params_data['time_advance']       = params_data['out_time_intvl']

params_data['batch_size']         = 64 * 8
params_data['shuffle_buffer']     = int(1e4)

# tfrecord dataset
DATA_DIR = './'
params_data['tfrecord_window_data'] = DATA_DIR+'tfrecord_window_'+str(params_data['in_time_intvl'])+'_'+str(params_data['out_time_intvl'])+'/'
params_data['num_unit_ae']          = 512

# Major slip positions
p4677_train_fail_index = np.load('./Data/p4677_train_fail_index.npy')
p4677_valid_fail_index = np.load('./Data/p4677_valid_fail_index.npy')
p4581_test_fail_index = np.load('./Data/p4581_3mpa_stepup_fail_index.npy')
params_data['slip_seg_size'] = 4

In [ ]:
# Parameters model
params_model = {}

params_model['num_vq_codes']        = 24 + 8
params_model['embedding_dim']       = int(params_data['unit_time_intvl']*params_data['sample_freq'])
params_model['commitment_cost']     = 0.25
params_model['ema_decay']           = 0.99

params_model['save_model_file']    = './Models/trsf_future_model-'+str(params_data['in_time_intvl'])+'-'+str(params_data['out_time_intvl'])
params_model['transformer_max_len'] = 1000
params_model['num_train_epochs']    = 500
params_model['learning_rate']       = 1e-3
params_model['lr_warmup_epochs']    = 10
params_model['lr_decay_epochs']     = 100
params_model['min_delta_stop']      = 1e-4
params_model['early_stop_wait']     = 10
params_model['verbose_model_train'] = 1

In [ ]:
params_data['num_in_time_intvl']  = 32
params_data['num_out_time_intvl'] = 1
params_data['in_time_intvl']      = params_data['unit_time_intvl'] * params_data['num_in_time_intvl']
params_data['out_time_intvl']     = params_data['unit_time_intvl'] * params_data['num_out_time_intvl']
params_data['in_out_time_diff']   = params_data['in_time_intvl']
params_data['time_advance']       = params_data['out_time_intvl']

params_data['tfrecord_window_data'] = DATA_DIR+'tfrecord_window_'+str(params_data['in_time_intvl'])+'_'+str(params_data['out_time_intvl'])+'/'

params_model['save_model_file']    = './Models/trsf_future_model-'+str(params_data['in_time_intvl'])+'-'+str(params_data['out_time_intvl'])

# Load Train/Valid datasets
train_dataset = read_tfrecord_dataset(params_data, 'train_data.tfrecord', is_shuffle=True)
valid_dataset = read_tfrecord_dataset(params_data, 'valid_data.tfrecord', is_shuffle=False)
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
train_dataset = train_dataset.with_options(options)
valid_dataset = valid_dataset.with_options(options)

train_data_size = np.load(params_data['tfrecord_window_data']+'train_dataset_size.npy')[0]

# Load Train Scaler
input_scaler_train = joblib.load(params_data['tfrecord_window_data']+'input_scaler_train.save')
output_scaler_train = joblib.load(params_data['tfrecord_window_data']+'output_scaler_train.save')
print(input_scaler_train.mean_,np.sqrt(input_scaler_train.var_))
print(output_scaler_train.data_min_, output_scaler_train.data_max_)
print('')

# Load Test Scaler
input_scaler_test = joblib.load(params_data['tfrecord_window_data']+'input_scaler_test.save')
output_scaler_test = joblib.load(params_data['tfrecord_window_data']+'output_scaler_test.save')
print(input_scaler_test.mean_,np.sqrt(input_scaler_test.var_))
print(output_scaler_test.data_min_, output_scaler_test.data_max_)

# Load model
labquake_model = tf.saved_model.load(params_model['save_model_file'])

In [ ]:
# Valid data
test_dataset = read_tfrecord_dataset(params_data, 'valid_data.tfrecord', is_shuffle=False)
input_scaler = input_scaler_train
output_scaler = output_scaler_train
fail_index = p4677_valid_fail_index

# Predict
targets, preds, targets_time, preds_slip, inputs, inputs_time, attn = test_labquake_model_detail(labquake_model, test_dataset, input_scaler, output_scaler)
mu_slip_loc_test, mu_slip_index = get_mu_slip_loc_data_with_index(params_data, targets, fail_index)
predict_signal = np.mean(preds, axis=1)
predict_signal_time = targets_time[:,int(8/2):int(8/2)+1]
target_signal = np.concatenate(targets, axis=0)
predict_signal = np.concatenate(predict_signal, axis=0)
target_signal_time = np.concatenate(targets_time, axis=0)
predict_signal_time = np.concatenate(predict_signal_time, axis=0)

# save all data for plot
predict_valid_1 = {}
predict_valid_1['inputs_time'] = inputs_time
predict_valid_1['inputs'] = inputs
predict_valid_1['predict_signal_time'] = predict_signal_time
predict_valid_1['predict_signal'] = predict_signal
predict_valid_1['target_signal_time'] = target_signal_time
predict_valid_1['target_signal'] = target_signal

In [ ]:
# Test data
test_dataset = read_tfrecord_dataset(params_data, 'test_data.tfrecord', is_shuffle=False)
input_scaler = input_scaler_test
output_scaler = output_scaler_test
fail_index = p4581_test_fail_index

# Predict
targets, preds, targets_time, preds_slip, inputs, inputs_time, attn = test_labquake_model_detail(labquake_model, test_dataset, input_scaler, output_scaler)
mu_slip_loc_test, mu_slip_index = get_mu_slip_loc_data_with_index(params_data, targets, fail_index)
predict_signal = np.mean(preds, axis=1)
predict_signal_time = targets_time[:,int(8/2):int(8/2)+1]
target_signal = np.concatenate(targets, axis=0)
predict_signal = np.concatenate(predict_signal, axis=0)
target_signal_time = np.concatenate(targets_time, axis=0)
predict_signal_time = np.concatenate(predict_signal_time, axis=0)

# save all data for plot
predict_test_1 = {}
predict_test_1['inputs_time'] = inputs_time
predict_test_1['inputs'] = inputs
predict_test_1['predict_signal_time'] = predict_signal_time
predict_test_1['predict_signal'] = predict_signal
predict_test_1['target_signal_time'] = target_signal_time
predict_test_1['target_signal'] = target_signal
predict_test_1['mu_slip_index_test'] = mu_slip_index
predict_test_1['preds_slip'] = preds_slip
predict_test_1['targets'] = targets
predict_test_1['targets_time'] = targets_time

In [ ]:
params_data['num_in_time_intvl']  = 32
params_data['num_out_time_intvl'] = 8
params_data['in_time_intvl']      = params_data['unit_time_intvl'] * params_data['num_in_time_intvl']
params_data['out_time_intvl']     = params_data['unit_time_intvl'] * params_data['num_out_time_intvl']
params_data['in_out_time_diff']   = params_data['in_time_intvl']
params_data['time_advance']       = params_data['out_time_intvl']

params_data['tfrecord_window_data'] = DATA_DIR+'tfrecord_window_'+str(params_data['in_time_intvl'])+'_'+str(params_data['out_time_intvl'])+'/'

params_model['save_model_file']    = './Models/trsf_future_model-'+str(params_data['in_time_intvl'])+'-'+str(params_data['out_time_intvl'])

# Load Train/Valid datasets
train_dataset = read_tfrecord_dataset(params_data, 'train_data.tfrecord', is_shuffle=True)
valid_dataset = read_tfrecord_dataset(params_data, 'valid_data.tfrecord', is_shuffle=False)
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
train_dataset = train_dataset.with_options(options)
valid_dataset = valid_dataset.with_options(options)

train_data_size = np.load(params_data['tfrecord_window_data']+'train_dataset_size.npy')[0]

# Load Train Scaler
input_scaler_train = joblib.load(params_data['tfrecord_window_data']+'input_scaler_train.save')
output_scaler_train = joblib.load(params_data['tfrecord_window_data']+'output_scaler_train.save')
print(input_scaler_train.mean_,np.sqrt(input_scaler_train.var_))
print(output_scaler_train.data_min_, output_scaler_train.data_max_)
print('')

# Load Test Scaler
input_scaler_test = joblib.load(params_data['tfrecord_window_data']+'input_scaler_test.save')
output_scaler_test = joblib.load(params_data['tfrecord_window_data']+'output_scaler_test.save')
print(input_scaler_test.mean_,np.sqrt(input_scaler_test.var_))
print(output_scaler_test.data_min_, output_scaler_test.data_max_)

# Load model
labquake_model = tf.saved_model.load(params_model['save_model_file'])

In [ ]:
# Valid data
test_dataset = read_tfrecord_dataset(params_data, 'valid_data.tfrecord', is_shuffle=False)
input_scaler = input_scaler_train
output_scaler = output_scaler_train
fail_index = p4677_valid_fail_index

# Predict
targets, preds, targets_time, preds_slip, inputs, inputs_time, attn = test_labquake_model_detail(labquake_model, test_dataset, input_scaler, output_scaler)
mu_slip_loc_test, mu_slip_index = get_mu_slip_loc_data_with_index(params_data, targets, fail_index)
predict_signal = np.mean(preds, axis=1)
predict_signal_time = targets_time[:,int(8/2):int(8/2)+1]
target_signal = np.concatenate(targets, axis=0)
predict_signal = np.concatenate(predict_signal, axis=0)
target_signal_time = np.concatenate(targets_time, axis=0)
predict_signal_time = np.concatenate(predict_signal_time, axis=0)

# save all data for plot
predict_valid_2 = {}
predict_valid_2['inputs_time'] = inputs_time
predict_valid_2['inputs'] = inputs
predict_valid_2['predict_signal_time'] = predict_signal_time
predict_valid_2['predict_signal'] = predict_signal
predict_valid_2['target_signal_time'] = target_signal_time
predict_valid_2['target_signal'] = target_signal

In [ ]:
# Test data
test_dataset = read_tfrecord_dataset(params_data, 'test_data.tfrecord', is_shuffle=False)
input_scaler = input_scaler_test
output_scaler = output_scaler_test
fail_index = p4581_test_fail_index

# Predict
targets, preds, targets_time, preds_slip, inputs, inputs_time, attn = test_labquake_model_detail(labquake_model, test_dataset, input_scaler, output_scaler)
mu_slip_loc_test, mu_slip_index = get_mu_slip_loc_data_with_index(params_data, targets, fail_index)
predict_signal = np.mean(preds, axis=1)
predict_signal_time = targets_time[:,int(8/2):int(8/2)+1]
target_signal = np.concatenate(targets, axis=0)
predict_signal = np.concatenate(predict_signal, axis=0)
target_signal_time = np.concatenate(targets_time, axis=0)
predict_signal_time = np.concatenate(predict_signal_time, axis=0)

# save all data for plot
predict_test_2 = {}
predict_test_2['inputs_time'] = inputs_time
predict_test_2['inputs'] = inputs
predict_test_2['predict_signal_time'] = predict_signal_time
predict_test_2['predict_signal'] = predict_signal
predict_test_2['target_signal_time'] = target_signal_time
predict_test_2['target_signal'] = target_signal
predict_test_2['mu_slip_index_test'] = mu_slip_index
predict_test_2['preds_slip'] = preds_slip
predict_test_2['targets'] = targets
predict_test_2['targets_time'] = targets_time

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(18, 20))
outer = gridspec.GridSpec(2, 1, wspace=0.0, hspace=0.23)

inner = gridspec.GridSpecFromSubplotSpec(3, 1,
                subplot_spec=outer[0], wspace=0.0, hspace=0.0)

ax1 = plt.Subplot(fig, inner[0])
ax1.plot(np.concatenate(predict_valid_1['inputs_time']), np.concatenate(predict_valid_1['inputs'])/1000.0, 'k-', linewidth=1.5, label='Data')
ax1.set_ylabel('Input \n scaled \n  |AE|', fontsize=28, color='k')
ax1.set_ylim([0.0, 2.0])
ax1.tick_params(axis='y', labelcolor='k')
fig.add_subplot(ax1)
ax1.set_xlim([2360, 2600.0])
ax1.set_xticks([])
ax1.tick_params(axis='y', width=2, length=10, color='black')
ax1.plot(0.0, 0.0, 'r--', linewidth=2, alpha=1, label='0.256s-0.008s')
ax1.plot(0.0, 0.0, 'b--', linewidth=2, alpha=1, label='0.256s-0.064s')
ax1.title.set_text('p4677 validation data')

ax2 = plt.Subplot(fig, inner[1])
ax2.plot(predict_valid_1['predict_signal_time'], predict_valid_1['predict_signal'], 'r--', linewidth=2, alpha=1, label='0.256s-0.008s')
ax2.plot(predict_valid_1['target_signal_time'], predict_valid_1['target_signal'], 'k-', linewidth=1.5, alpha=1)
fig.add_subplot(ax2)
ax2.set_xlim([2360, 2600.0])
ax2.set_ylabel('Output $\mu$', color='r')
ax2.tick_params(axis='y', labelcolor='r')
ax2.set_yticks([0.45, 0.55, 0.65])
ax2.set_ylim([0.42, 0.7])
ax2.set_xticks([])
ax2.tick_params(axis='y', width=2, length=10, color='red')

ax3 = plt.Subplot(fig, inner[2])
ax3.plot(predict_valid_2['predict_signal_time'], predict_valid_2['predict_signal'], 'b--', linewidth=2, alpha=1, label='0.256s-0.064s')
ax3.plot(predict_valid_2['target_signal_time'], predict_valid_2['target_signal'], 'k-', linewidth=1.5, alpha=1)
fig.add_subplot(ax3)
ax3.set_xlim([2360, 2600.0])
ax3.set_xticks([2400, 2450, 2500, 2550, 2600])
ax3.set_ylabel('Output $\mu$', color='b')
ax3.set_xlabel('Time [s]')
ax3.tick_params(axis='y', labelcolor='b')
ax3.set_yticks([0.45, 0.55, 0.65])
ax3.set_ylim([0.42, 0.7])
ax3.tick_params(axis='y', width=2, length=10, color='blue')
ax3.tick_params(axis='x', width=2, length=10, color='black')

ax1.legend(loc='upper right', fontsize=20)

inner = gridspec.GridSpecFromSubplotSpec(3, 1,
                subplot_spec=outer[1], wspace=0.0, hspace=0.0)

ax1 = plt.Subplot(fig, inner[0])
ax1.plot(np.concatenate(predict_test_1['inputs_time']), np.concatenate(predict_test_1['inputs'])/1000.0, 'k-', linewidth=1.5, label='Data')
fig.add_subplot(ax1)
ax1.set_xlim([2325, 2525.0])
ax1.set_xticks([])
ax1.set_ylabel('Input \n scaled \n  |AE|', fontsize=28, color='k')
ax1.set_xlabel('Time [s]', fontsize=28)
ax1.set_ylim([0.0, 2.0])
ax1.tick_params(axis='y', width=2, length=10, color='black')
ax1.title.set_text('p4581 testing data')

ax2 = plt.Subplot(fig, inner[1])
ax2.plot(predict_test_1['predict_signal_time'], predict_test_1['predict_signal'], 'r--', linewidth=2, alpha=1, label='0.256s-0.008s')
ax2.plot(predict_test_1['target_signal_time'], predict_test_1['target_signal'], 'k-', linewidth=1.5, alpha=1)
fig.add_subplot(ax2)
ax2.set_xlim([2325, 2525.0])
ax2.set_xticks([])
ax2.set_ylim([0.32, 0.48])
ax2.set_ylabel('Output $\mu$', color='r')
ax2.tick_params(axis='y', labelcolor='r')
ax2.set_yticks([0.35, 0.40, 0.45])
ax2.tick_params(axis='y', width=2, length=10, color='red')

ax3 = plt.Subplot(fig, inner[2])
ax3.plot(predict_test_2['predict_signal_time'], predict_test_2['predict_signal'], 'b--', linewidth=2, alpha=1, label='0.256s-0.064s')
ax3.plot(predict_test_2['target_signal_time'], predict_test_2['target_signal'], 'k-', linewidth=1.5, alpha=1)
fig.add_subplot(ax3)
ax3.set_xlim([2325, 2525.0])
ax3.set_ylim([0.32, 0.48])
ax3.set_ylabel('Output $\mu$', color='b')
ax3.tick_params(axis='y', labelcolor='b')
ax3.set_yticks([0.35, 0.40, 0.45])
ax3.set_xticks([2325, 2350, 2375, 2400, 2425, 2450, 2475, 2500, 2525])
ax3.set_xlabel('Time [s]')
ax3.tick_params(axis='x', width=2, length=10, color='black')
ax3.tick_params(axis='y', width=2, length=10, color='blue')

mu_slip_index_test = predict_test_1['mu_slip_index_test']
targets_time = predict_test_1['targets_time']

mu_slip_index_sub = np.copy(mu_slip_index_test)
mu_slip_index_sub = np.delete(mu_slip_index_sub, 13)

letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']

for ii, let in zip(mu_slip_index_sub, letters):
  start_time = targets_time[ii][0]
  end_time = start_time + 0.008

  ax2.text(start_time+0.1, 0.33, let, fontsize='28')
  x_time = np.arange(start_time, end_time, 0.001)
  ax2.fill_between(x_time, np.ones_like(x_time)*0.3,  np.ones_like(x_time)*0.5, color='red', alpha=0.5)

fig.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

preds_slip = predict_test_1['preds_slip']
targets = predict_test_1['targets']
mu_slip_index = predict_test_1['mu_slip_index_test']
out_time = 0.0 + np.arange(0,8,1)/1000.0

mu_slip_index_sub = np.copy(mu_slip_index)
mu_slip_index_sub = np.delete(mu_slip_index_sub, 13)
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']

fig = plt.figure(figsize=(20, 20))
outer = gridspec.GridSpec(5, 5, wspace=0.05, hspace=0.05)

x_time = np.arange(0.0, 0.009, 0.001)
y_low = np.ones_like(x_time)*0.35
y_up = np.ones_like(x_time)*0.47

for i in range(len(mu_slip_index)):
    ax3 = plt.Subplot(fig, outer[i])
    if i == 0:
        ax3.plot(out_time, preds_slip[mu_slip_index[i],:,:], 'r--', linewidth=5, alpha=1, label='Prediction')
        ax3.plot(out_time, targets[mu_slip_index[i],:,:], 'k-', linewidth=4, alpha=1, label='Data')
    else:
        ax3.plot(out_time, preds_slip[mu_slip_index[i],:,:], 'r--', linewidth=5, alpha=1)
        ax3.plot(out_time, targets[mu_slip_index[i],:,:], 'k-', linewidth=4, alpha=1)
    ax3.tick_params(axis='y', labelcolor='r')
    ax3.set_ylim([0.35, 0.47])
    ax3.set_xlim([0.0, 0.008])
    if i == 0:
        ax3.set_ylabel('Output $\mu$', color='r', fontsize=30)
        ax3.set_yticks([0.35, 0.40, 0.45], fontsize=30)
        ax3.set_xticks([0.0, 0.004, 0.008], fontsize=30)
        ax3.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
        ax3.set_xlabel('Time [s]', fontsize='30')
        ax3.xaxis.set_ticks_position('top')
        ax3.xaxis.set_label_position('top')
        ax3.tick_params(axis='x', labelsize=30, width=5, length=10, color='black')
        ax3.tick_params(axis='y', labelsize=30, width=5, length=10, color='red')
        ax3.xaxis.set_label_coords(.5, 1.3)
        ax3.legend(bbox_to_anchor=(3, -2.5), loc='upper left', borderaxespad=0, fontsize=30)
    else:
        ax3.set_yticks([])
        ax3.set_xticks([])
    
    if mu_slip_index[i] in mu_slip_index_sub:
        ax3.fill_between(x_time, y_low,  y_up, color='red', alpha=0.05)
        ind = np.where(mu_slip_index_sub == mu_slip_index[i])
        ax3.text(0.0005, 0.36, letters[ind[0][0]], fontsize=30)
    fig.add_subplot(ax3)

fig.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

preds_slip = predict_test_2['preds_slip']
targets = predict_test_2['targets']
mu_slip_index = predict_test_2['mu_slip_index_test']
out_time = 0.0 + np.arange(0,64,1)/1000.0

mu_slip_index_sub = np.copy(mu_slip_index)
mu_slip_index_sub = np.delete(mu_slip_index_sub, 13)
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']

fig = plt.figure(figsize=(20, 20))
outer = gridspec.GridSpec(5, 5, wspace=0.05, hspace=0.05)

x_time = np.arange(0.0, 0.065, 0.001)
y_low = np.ones_like(x_time)*0.35
y_up = np.ones_like(x_time)*0.47

for i in range(len(mu_slip_index)):
    ax3 = plt.Subplot(fig, outer[i])
    if i == 0:
        ax3.plot(out_time, preds_slip[mu_slip_index[i],:,:], 'r--', linewidth=5, alpha=1, label='Prediction')
        ax3.plot(out_time, targets[mu_slip_index[i],:,:], 'k-', linewidth=4, alpha=1, label='Data')
    else:
        ax3.plot(out_time, preds_slip[mu_slip_index[i],:,:], 'r--', linewidth=5, alpha=1)
        ax3.plot(out_time, targets[mu_slip_index[i],:,:], 'k-', linewidth=4, alpha=1)
    ax3.tick_params(axis='y', labelcolor='r')
    ax3.set_ylim([0.35, 0.47])
    ax3.set_xlim([0.0, 0.064])
    if i == 0:
        ax3.set_ylabel('Output $\mu$', color='r', fontsize=30)
        ax3.set_yticks([0.35, 0.40, 0.45], fontsize=30)
        ax3.set_xticks([0.0, 0.032, 0.064], fontsize=30)
        ax3.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
        ax3.set_xlabel('Time [s]', fontsize='30')
        ax3.xaxis.set_ticks_position('top')
        ax3.xaxis.set_label_position('top')
        ax3.tick_params(axis='x', labelsize=30, width=5, length=10, color='black')
        ax3.tick_params(axis='y', labelsize=30, width=5, length=10, color='red')
        ax3.xaxis.set_label_coords(.5, 1.3)
        ax3.legend(bbox_to_anchor=(3, -2.5), loc='upper left', borderaxespad=0, fontsize=30)
    else:
        ax3.set_yticks([])
        ax3.set_xticks([])
    
    fig.add_subplot(ax3)

fig.show()